In [1]:
source activate qiime2-2022.2

(qiime2-2022.2) 

: 1

# Importing and Trimming Data #

In [4]:
qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]'\
    --input-path DSS1_manifest.csv \
    --output-path paired-end-demux.qza \
    --input-format PairedEndFastqManifestPhred33

#Importing data from fastq files
#usually we would need to demultiplex but since we are using the manifest, this is already done
#Phred33 refers to the cutoff of sequence quality, which we are using as 33

Imported DSS1_manifest.csv as PairedEndFastqManifestPhred33 to paired-end-demux.qza
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [6]:
qiime demux summarize \
--i-data paired-end-demux.qza \
--o-visualization paired-end-demux.qzv

Saved Visualization to: paired-end-demux.qzv
(qiime2-2022.2) 

: 1

In [3]:
qiime tools view paired-end-demux.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

# DADA2 #

In [3]:
qiime dada2 denoise-paired \
--i-demultiplexed-seqs paired-end-demux.qza \
--o-table table.qza \
--o-representative-sequences rep-seqs.qza \
--p-trim-left-f 6 \
--p-trim-left-r 6 \
--p-trunc-len-f 247 \
--p-trunc-len-r 190 \
--p-min-fold-parent-over-abundance 5 \
--p-n-threads 0 \
--o-denoising-stats stats-dada2.qza \
--verbose

#using our demultiplexed sequences and info from our interactive quality plot, see where quality drops off 
#for both forward and reverse, at the beginning and end, and cut there
#originally had a high proportion of chimeras, even after trimming primers off
#changed chimera stringency in dada2 using the p-min fold over abundance to 5 from default 2

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_paired.R /tmp/tmp4z_04a88/forward /tmp/tmp4z_04a88/reverse /tmp/tmp4z_04a88/output.tsv.biom /tmp/tmp4z_04a88/track.tsv /tmp/tmp4z_04a88/filt_f /tmp/tmp4z_04a88/filt_r 300 271 0 0 2.0 2.0 2 12 independent consensus 5.0 0 1000000

R version 4.0.5 (2021-03-31) 
Loading required package: Rcpp
DADA2: 1.18.0 / Rcpp: 1.0.7 / RcppParallel: 5.1.4 
1) Filtering Error in names(answer) <- names1 : 
  'names' attribute [171] must be the same length as the vector [149]
Execution halted
Traceback (most recent call last):
  File "/home/etb/anaconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/q2_dada2/_denoise.py", line 266, in denoise_paired
    run_commands([cmd])
  File "/home/etb/anaconda3/envs/qiime2-2021.8/lib/python3.8/site-packages/q2_dada2/_d

: 1

In [3]:
qiime metadata tabulate \
--m-input-file stats-dada2.qza \
--o-visualization stats-dada2.qzv

#An important check here to see how many reads passed through each dada2 step.
#Unless quality is poor, should have over 75% of reads at the end.
#otherwise-- issue with sequencing run, primer removal etc.

Saved Visualization to: stats-dada2.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [4]:
qiime tools view stats-dada2.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

# Feature Table

In [5]:
qiime feature-table summarize \
    --i-table table.qza \
    --o-visualization table.qzv \
    --m-sample-metadata-file DSS1_metadata.tsv

#creating a visual summary of the data
#inputting the metadata file here - associating a time point, treatment group, etc to each sample

Saved Visualization to: table.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [19]:
qiime tools view table.qzv 

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

# Representative Sequences

In [12]:
qiime feature-table tabulate-seqs \
    --i-data rep-seqs.qza \
    --o-visualization rep-seqs.qzv

Saved Visualization to: rep-seqs.qzv
(qiime2-2022.2) 

: 1

In [10]:
qiime tools view rep-seqs.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

# Making A Tree and Performing Diversity Analyses #

In [13]:
qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences rep-seqs.qza \
    --o-alignment aligned-rep-seqs.qza \
    --o-masked-alignment masked-aligned-rep-seqs.qza \
    --o-tree unrooted-tree.qza \
    --o-rooted-tree rooted-tree.qza 

#For the diversity metrics that rely on phylogeny ie. UniFrac, maaft performs a multiple alignment
#Next the pipeline removes highly variable positions to mask these
#Then the fasttree program creates a tree

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

## All cages

In [15]:
qiime diversity core-metrics-phylogenetic \
--i-phylogeny rooted-tree.qza \
--i-table table.qza \
--p-sampling-depth 4880 \
--m-metadata-file DSS1_metadata.tsv \
  --o-rarefied-table rarefied-table.qza \
  --o-faith-pd-vector faith-pd.qza \
  --o-observed-features-vector observed-otus.qza \
  --o-shannon-vector shannon-vector.qza \
  --o-evenness-vector evenness-vector.qza \
  --o-unweighted-unifrac-distance-matrix unweighted-unifrac-distance-matrix.qza \
  --o-weighted-unifrac-distance-matrix weighted-unifrac-distance-matrix.qza \
  --o-jaccard-distance-matrix jaccard-distance-matrix.qza \
  --o-bray-curtis-distance-matrix bray-curtis-distance-matrix.qza \
  --o-unweighted-unifrac-pcoa-results unweighted-unifrac-pca-results.qza \
  --o-weighted-unifrac-pcoa-results weighted-unifrac-pcoa-results.qza \
  --o-jaccard-pcoa-results jaccard-pcoa-results.qza \
  --o-bray-curtis-pcoa-results bray-curtis-pcoa-results.qza \
  --o-unweighted-unifrac-emperor unweighted-unifrac-emperor.qzv \
  --o-weighted-unifrac-emperor weighted-unifrac-emperor.qzv \
  --o-jaccard-emperor jaccard-emperor.qzv \
  --o-bray-curtis-emperor bray-curtis-emperor.qzv

# Here doing diversity analyses for the table with sheath fluid samples but not the sequencing controls
#qiime diversity computes several alpha and beta diversity metrics after sampling depth is rarefied. 
#For beta diversity, several of these can be visualized using PcoA and emperor
#For purposes of stats, important to use the distance matrices that were generated

Saved FeatureTable[Frequency] to: rarefied-table.qza
Saved SampleData[AlphaDiversity] to: faith-pd.qza
Saved SampleData[AlphaDiversity] to: observed-otus.qza
Saved SampleData[AlphaDiversity] to: shannon-vector.qza
Saved SampleData[AlphaDiversity] to: evenness-vector.qza
Saved DistanceMatrix to: unweighted-unifrac-distance-matrix.qza
Saved DistanceMatrix to: weighted-unifrac-distance-matrix.qza
Saved DistanceMatrix to: jaccard-distance-matrix.qza
Saved DistanceMatrix to: bray-curtis-distance-matrix.qza
Saved PCoAResults to: unweighted-unifrac-pca-results.qza
Saved PCoAResults to: weighted-unifrac-pcoa-results.qza
Saved PCoAResults to: jaccard-pcoa-results.qza
Saved PCoAResults to: bray-curtis-pcoa-results.qza
Saved Visualization to: unweighted-unifrac-emperor.qzv
Saved Visualization to: weighted-unifrac-emperor.qzv
Saved Visualization to: jaccard-emperor.qzv
Saved Visualization to: bray-curtis-emperor.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (q

: 1

# Viewing Beta Diversity Metrics (all data)

In [20]:
qiime tools view weighted-unifrac-emperor.qzv

# loose grouping by sex/cage & health state
# F1 & F2 are distinct from each other, but the two male cages seem similar

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

In [21]:
qiime tools view unweighted-unifrac-emperor.qzv

# grouping by sex/cage & health state
# F1 & F2 are distinct from each other, but the two male cages seem similar

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

In [22]:
qiime tools view jaccard-emperor.qzv

# grouping by cage & sex; loose grouping by health state
# F1 & F2 are distinct from each other, but the two male cages seem similar

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

In [23]:
qiime tools view bray-curtis-emperor.qzv

# horseshoe-shaped
# grouping by cage & sex; very loose grouping by health state
# F1 & F2 are distinct from each other, but the two male cages seem similar

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

# Beta Diversity Significance Analyses (all data)

## Weighted Unifrac

In [48]:
### beta-group significance on table vectors ####

qiime diversity beta-group-significance \
--i-distance-matrix  weighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column sex \
--o-visualization wunifrac-sex-beta-sig.qzv \
--p-pairwise

# comparing between the two sexes

(qiime2-2022.2) (qiime2-2022.2) Saved Visualization to: wunifrac-sex-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view wunifrac-sex-beta-sig.qzv

In [49]:
qiime diversity beta-group-significance \
--i-distance-matrix  weighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column cage \
--o-visualization wunifrac-cage-beta-sig.qzv \
--p-pairwise

# comparing between cages

Saved Visualization to: wunifrac-cage-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view wunifrac-cage-beta-sig.qzv

In [50]:
qiime diversity beta-group-significance \
--i-distance-matrix  weighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column health-state \
--o-visualization wunifrac-hs-beta-sig.qzv \
--p-pairwise

# comparing between health states (Pre-DSS, DSS, Recovery, Follow-up)

Saved Visualization to: wunifrac-hs-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view wunifrac-hs-beta-sig.qzv

In [52]:
qiime diversity beta-group-significance \
--i-distance-matrix  weighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column day \
--o-visualization wunifrac-day-beta-sig.qzv \
--p-pairwise

# comparing between days

Saved Visualization to: wunifrac-day-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view wunifrac-day-beta-sig.qzv

## Unweighted Unifrac

In [53]:
qiime diversity beta-group-significance \
--i-distance-matrix  unweighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column sex \
--o-visualization un-unifrac-sex-beta-sig.qzv \
--p-pairwise

# comparing between the two sexes

Saved Visualization to: un-unifrac-sex-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view un-unifrac-sex-beta-sig.qzv

In [54]:
qiime diversity beta-group-significance \
--i-distance-matrix  unweighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column cage \
--o-visualization un-unifrac-cage-beta-sig.qzv \
--p-pairwise

# comparing between cages

Saved Visualization to: un-unifrac-cage-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view un-unifrac-cage-beta-sig.qzv

In [55]:
qiime diversity beta-group-significance \
--i-distance-matrix  unweighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column health-state \
--o-visualization un-unifrac-hs-beta-sig.qzv \
--p-pairwise

# comparing between health states

Saved Visualization to: un-unifrac-hs-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view un-unifrac-hs-beta-sig.qzv

In [56]:
qiime diversity beta-group-significance \
--i-distance-matrix  unweighted-unifrac-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column day \
--o-visualization un-unifrac-day-beta-sig.qzv \
--p-pairwise

# comparing between days

Saved Visualization to: un-unifrac-day-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view un-unifrac-day-beta-sig.qzv

## Jaccard

In [57]:
qiime diversity beta-group-significance \
--i-distance-matrix  jaccard-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column sex \
--o-visualization jaccard-sex-beta-sig.qzv \
--p-pairwise

# comparing between the two sexes

Saved Visualization to: jaccard-sex-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view jaccard-sex-beta-sig.qzv

In [58]:
qiime diversity beta-group-significance \
--i-distance-matrix  jaccard-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column cage \
--o-visualization jaccard-cage-beta-sig.qzv \
--p-pairwise

# comparing between cages

Saved Visualization to: jaccard-cage-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view jaccard-cage-beta-sig.qzv

In [59]:
qiime diversity beta-group-significance \
--i-distance-matrix  jaccard-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column health-state \
--o-visualization jaccard-hs-beta-sig.qzv \
--p-pairwise

# comparing between health states

Saved Visualization to: jaccard-hs-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view jaccard-hs-beta-sig.qzv

In [60]:
qiime diversity beta-group-significance \
--i-distance-matrix  jaccard-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column day \
--o-visualization jaccard-day-beta-sig.qzv \
--p-pairwise

# comparing between days

Saved Visualization to: jaccard-day-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view jaccard-day-beta-sig.qzv

## Bray-Curtis

In [61]:
qiime diversity beta-group-significance \
--i-distance-matrix bray-curtis-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column sex \
--o-visualization bc-sex-beta-sig.qzv \
--p-pairwise

# comparing between the two sexes

Saved Visualization to: bc-sex-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view bc-sex-beta-sig.qzv

In [62]:
qiime diversity beta-group-significance \
--i-distance-matrix bray-curtis-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column cage \
--o-visualization bc-cage-beta-sig.qzv \
--p-pairwise

# comparing between cages

Saved Visualization to: bc-cage-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view bc-cage-beta-sig.qzv

In [63]:
qiime diversity beta-group-significance \
--i-distance-matrix bray-curtis-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column health-state \
--o-visualization bc-hs-beta-sig.qzv \
--p-pairwise

# comparing between health states

Saved Visualization to: bc-hs-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view bc-hs-beta-sig.qzv

In [64]:
qiime diversity beta-group-significance \
--i-distance-matrix bray-curtis-distance-matrix.qza  \
--m-metadata-file DSS1_metadata.tsv \
--m-metadata-column day \
--o-visualization bc-day-beta-sig.qzv \
--p-pairwise

# comparing between days

Saved Visualization to: bc-day-beta-sig.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [ ]:
qiime tools view bc-day-beta-sig.qzv

# Alpha diversity significance analyses (all data)

In [74]:
qiime diversity alpha-group-significance \
--i-alpha-diversity shannon-vector.qza \
--m-metadata-file DSS1_metadata.tsv \
--o-visualization shannon-alpha-diversity.qzv 

Saved Visualization to: shannon-alpha-diversity.qzv
(qiime2-2022.2) 

: 1

In [58]:
qiime tools view shannon-alpha-diversity.qzv 

# ns btwn health states, cages, sex

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

In [77]:
qiime diversity alpha \
--i-table table.qza \
--p-metric simpson \
--o-alpha-diversity simpson-index.qza 

Saved SampleData[AlphaDiversity] to: simpson-index.qza
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [78]:
qiime diversity alpha-group-significance \
--i-alpha-diversity simpson-index.qza \
--m-metadata-file DSS1_metadata.tsv \
--o-visualization simpson-index.qzv 

Saved Visualization to: simpson-index.qzv
(qiime2-2022.2) 

: 1

In [59]:
qiime tools view simpson-index.qzv 

# Pre-DSS (higher) vs DSS (lower) q = 0.028015
# ns between cages, sex

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

In [79]:
qiime diversity alpha \
--i-table table.qza \
--p-metric simpson_e \
--o-alpha-diversity simpsonE-index.qza 

# simpson EVENNESS which accounts only for evenness and not abundance

Saved SampleData[AlphaDiversity] to: simpsonE-index.qza
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [80]:
qiime diversity alpha-group-significance \
--i-alpha-diversity simpsonE-index.qza \
--m-metadata-file DSS1_metadata.tsv \
--o-visualization simpsonE-index.qzv 

Saved Visualization to: simpsonE-index.qzv
(qiime2-2022.2) 

: 1

In [60]:
qiime tools view simpsonE-index.qzv 

# Pre-DSS vs DSS q = 0.019409 (DSS lower)
# ns btwn cage, sex

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

In [81]:
qiime diversity alpha-group-significance \
--i-alpha-diversity observed-otus.qza \
--m-metadata-file DSS1_metadata.tsv \
--o-visualization observed-otus.qzv 

Saved Visualization to: observed-otus.qzv
(qiime2-2022.2) 

: 1

In [61]:
qiime tools view observed-otus.qzv 

# ns btwn health states, cage, sex

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

In [27]:
qiime diversity alpha-group-significance \
--i-alpha-diversity evenness-vector.qza \
--m-metadata-file DSS1_metadata.tsv \
--o-visualization evenness-vector.qzv 

Saved Visualization to: evenness-vector.qzv
(qiime2-2022.2) 

: 1

In [62]:
qiime tools view evenness-vector.qzv 

# Pielou's evenness (idk what that takes into account)
# https://www.marinespecies.org/introduced/wiki/Measurements_of_biodiversity#Pielou_index

# Pre-DSS vs DSS (lower) q = 0.010936
# Pre-DSS vs Colitis (lower) q = 0.032011

# ns between cage, sex (F2 seems quite different from the rest, though)

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 

# Classifier & Taxonomy #

In [7]:
# trains a classifier on the Silva database for V4-5 region (515F, 926R)

#Extract reference reads:
qiime feature-classifier extract-reads \
--i-sequences silva-138-99-seqs.qza \
--p-f-primer GTGYCAGCMGCCGCGGTAA \
--p-r-primer CCGYCAATTYMTTTRAGTTT \
--p-trunc-len 0 \
--p-min-length 100 \
--p-max-length 450 \
--o-reads ref-seqs-v138-V4V5.qza

(qiime2-2021.4) (qiime2-2021.4) (qiime2-2021.4) Saved FeatureData[Sequence] to: ref-seqs-v138-V4V5.qza
(qiime2-2021.4) 

: 1

In [9]:
#Train the classifier (needs about 20 GB memory from my experience):

qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads ref-seqs-v138-V4V5.qza \
--i-reference-taxonomy silva-138-99-tax.qza \
--o-classifier silva-138-V4V5-classifier.qza

(qiime2-2021.4) Killed
(qiime2-2021.4) 

: 1

In [5]:
#Get the taxonomy of your sequences using the trained classifier (needs about 6-10 Gbs using 8 threads)

qiime feature-classifier classify-sklearn \
--i-classifier silva-138-V4V5-classifier.qza \
--i-reads rep-seqs.qza \
--o-classification taxonomy.qza \
--p-n-jobs 8

#--verbose \
#--p-reads-per-batch 1000

Killed
(qiime2-2020.8) 

: 1

In [16]:
# The taxonomy.qza file relates ASVs to taxonomic classification & gives % confidence of classifications
# we generate a visualization of this file as below

qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) Saved Visualization to: taxonomy.qzv
(qiime2-2022.2) 

: 1

In [4]:
qiime tools view taxonomy.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2021.11) 

In [17]:
qiime taxa barplot \
--i-table table.qza \
--i-taxonomy taxonomy.qza \
--m-metadata-file DSS1_metadata.tsv \
--o-visualization taxa-bar-plots.qzv

#using the qiime taxa bar plots command to generate bar plots with various taxonomy levels 
#can sort by metadata but visualization isnt great with so many samples 
#grouped bar plots and longitudinal - if time series data - can help with this 

Saved Visualization to: taxa-bar-plots.qzv
(qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) (qiime2-2022.2) 

: 1

In [18]:
qiime tools view taxa-bar-plots.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.(qiime2-2022.2) 